# Import everything

In [ ]:
import csv
import torch
from tqdm import tqdm
from settings import cfg
from helper import loader, boilerplate
from settings.cfg import model
from settings.cfg import batch_size, tests_paths
from settings.cfg import model_path, result_path

# Get the dataset


In [1]:
test_image, test_names = loader.load_data(tests_paths)

# Boilerplate Code

In [2]:
Data = boilerplate.ExportDataset

# Model

In [3]:
model.load_state_dict(torch.load(model_path))


# Run tests

In [4]:
tests = Data(test_image, test_names)
tests_dataloader = torch.utils.data.DataLoader(tests, batch_size=batch_size)

with torch.no_grad(), open(cfg.result_path, "w", newline='') as result:
    result_writer = csv.writer(result)
    result_writer.writerow(["Frame", "Label"])
    for image, names in tqdm(tests_dataloader):
        image = image.to(cfg.device, dtype=torch.float)

        output = model(image)
        output = output.argmax(dim=1).cpu().numpy()
        assert(output.shape[0] == len(names))
        
        result_writer.writerows(zip(names, output))


100%|██████████| 37/37 [00:11<00:00,  3.22it/s]


# Submit


In [5]:
from datetime import datetime
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

api.competition_submit(config.result_path, f"Submitted at {datetime.now()}", "hsgs-hackathon2022")


100%|██████████| 238k/238k [00:03<00:00, 64.5kB/s] 


Successfully submitted to HSGS Hackathon2022